In [ ]:
# ==================================================================================================================
# ===========================================> Instalación librerías <==============================================
# ==================================================================================================================

#%pip install pandas    # NO HACE FALTA INSTALAR
#%pip install numpy     # NO HACE FALTA INSTALAR
#%pip install pickle    # NO HACE FALTA INSTALAR
#%pip install sklearn   # NO HACE FALTA INSTALAR

#%pip install scikit-learn          # DESCOMENTAR LA PRIMERA VEZ
#%pip install category_encoders     # DESCOMENTAR LA PRIMERA VEZ
#%pip install lightgbm              # DESCOMENTAR LA PRIMERA VEZ
#%pip install matplotlib            # DESCOMENTAR LA PRIMERA VEZ
#%pip install seaborn               # DESCOMENTAR LA PRIMERA VEZ



In [2]:
# ==================================================================================================================
# ===========================================> Carga librerías <====================================================
# ==================================================================================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# ==================================================================================================================
# ===========================================> Carga Datos <========================================================
# ==================================================================================================================

df = pd.read_csv('data/train.csv', low_memory=False)

# =====================> EDA

In [4]:
# ==================================================================================================================
# ===========================================> Exploración número de columnas <=====================================
# ==================================================================================================================

len(df.columns)

83

In [5]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col numericas <=====================
# ==================================================================================================================

df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,Firewall,UacLuaenable,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,892148.000000,888846.000000,892148.000000,43369.000000,888564.000000,888564.000000,888564.000000,892148.000000,892148.000000,859124.000000,616605.00000,892131.000000,892148.000000,892148.000000,892148.000000,888586.000000,892148.000000,838350.000000,886309.000000,883049.000000,890987.000000,882496.000000,881821.000000,887964.000000,887964.000000,887959.000000,8.868130e+05,8.868160e+05,892148.000000,884057.000000,887556.000000,887568.000000,887568.000000,8.650860e+05,892148.000000,892148.000000,886071.000000,892148.000000,892148.000000,151099.000000,876198.000000,325283.000000,873771.000000,876065.000000,892148.000000,326030.0,890504.000000,892148.000000,892148.000000,884915.000000,861721.000000,861721.00000,892148.000000
mean,0.000006,6.844869,0.017356,1661.352717,47837.854700,1.326307,1.020843,0.988129,107.994099,81322.998282,24.86081,169.596720,122.829220,15721.197375,575.094119,0.945629,0.000025,0.000493,126.553567,0.978359,0.994819,2222.020415,239080.384829,3.990466,4.530417,2370.803384,5.132934e+05,3.770886e+05,0.077197,6120.538462,16.679802,1547.741835,897.689508,1.127727e+09,15834.761598,971.799178,14.610311,60.463321,0.000562,0.000013,0.000007,0.000218,402.581156,33031.078203,0.485480,0.0,0.006984,0.125847,0.038329,0.057437,0.282957,7.88412,0.499793
std,0.002367,1.027253,0.130593,1002.801316,14038.111979,0.522860,0.167380,0.108307,63.047616,48932.357294,5.61479,89.334425,69.284457,2189.357659,248.097295,0.226748,0.004966,0.022190,42.620373,0.145508,0.207985,1316.242986,72014.597677,2.080902,1.288213,840.755233,3.661301e+05,3.240502e+05,0.266904,5119.763016,5.894300,368.563317,214.878100,1.889917e+09,1961.975542,2927.748772,10.194424,44.976600,0.023691,0.003638,0.002617,0.014772,221.677719,21209.367078,0.499789,0.0,0.083276,0.331677,0.191989,0.232676,0.450436,4.55187,0.500000
min,0.000000,0.000000,0.000000,1.000000,7.000000,1.000000,0.000000,0.000000,1.000000,5.000000,1.00000,1.000000,1.000000,7600.000000,16.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000e+00,0.000000e+00,0.000000,400.000000,3.300000,-1.000000,-1.000000,0.000000e+00,9200.000000,0.000000,1.000000,5.000000,0.000000,0.000000,0.000000,0.000000,11.000000,12.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000
25%,0.000000,7.000000,0.000000,788.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,36825.000000,18.00000,89.000000,75.000000,15063.000000,256.000000,1.000000,0.000000,0.000000,111.000000,1.000000,1.000000,1443.000000,189692.000000,2.000000,5.000000,1998.000000,2.393720e+05,1.207650e+05,0.000000,4096.000000,13.900000,1366.000000,768.000000,0.000000e+00,15063.000000,167.000000,8.000000,31.000000,0.0000

In [6]:
# ==================================================================================================================
# ===========================================> Exploración estadisticas basicas col objectos <=====================
# ==================================================================================================================

df.describe(include=object)

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,OsBuildLab,SkuEdition,PuaMode,SmartScreen,Census_MDC2FormFactor,Census_DeviceFamily,Census_ProcessorClass,Census_PrimaryDiskTypeName,Census_ChassisTypeName,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
count,892148,892148,892148,892148,892148,892148,892148,892148,892148,892148,892148,223,574786,892148,892148,3731,890851,892079,892142,258161,892148,892148,892148,892148,892148,892148,892148,892148,892148,892148
unique,892148,5,55,95,7242,4,3,20,9,494,8,1,11,12,3,3,4,39,9,37,328,3,17,24,20,9,6,4,6,8
top,7bd132f88d90f8c628ae7204ce1e0038,win8defender,1.1.15200.1,4.18.1807.18075,1.273.1420.0,windows10,x64,10.0.0.0,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,on,RequireAdmin,Notebook,Windows.Desktop,mid,HDD,Notebook,Mobile,lion,10.0.17134.228,amd64,rs4_release,Core,CORE,UUPUpgrade,FullAuto,IS_GENUINE,Retail,Retail
freq,1,882704,384906,514198,10277,861966,810681,863355,392050,366301,551358,223,431685,571581,890692,2081,580101,524416,617432,202757,141486,810750,401270,347418,347402,260597,395760,787219,473562,835306


In [7]:
# ==================================================================================================================
# ===========================================> Exploración 20 primeras filas <======================================
# ==================================================================================================================

df.head(20)

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,7bd132f88d90f8c628ae7204ce1e0038,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1486.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,51,103807.0,27.0,98.0,103,windows10,x64,10.0.0.0,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Home,1.0,0,NaN,0.0,117.0,ExistsNotSet,1.0,1.0,Notebook,Windows.Desktop,2668.0,171473.0,4.0,5.0,2697.0,NaN,953869.0,HDD,953253.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,NaN,0.000000e+00,10.0.16299.547,amd64,rs3_release_svc_escrow,16299,547,Core,CORE,IBSClean,5.0,26,UNKNOWN,0,IS_GENUINE,OEM:DM,NaN,0.0,Retail,NaN,628.0,12300.0,1,NaN,0.0,0,0,0.0,0.0,6.0,1
1,12678cb674ee9f2c85e526dc9774845d,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1234.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,35,27424.0,27.0,45.0,83,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,525.0,331262.0,2.0,5.0,1985.0,NaN,476940.0,HDD,190774.0,0,4096.0,Notebook,15.5,1366.0,768.0,Mobile,NaN,0.000000e+00,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,Reset,13.0,48,FullAuto,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,142.0,70187.0,0,NaN,0.0,0,0,0.0,1.0,4.0,0
2,624400f847781fe7ccc3546e921fb46f,win8defender,1.1.15000.2,4.18.1806.18062,1.271.545.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,211,10989.0,18.0,277.0,75,windows10,x64,10.0.0.0,16299,768,rs3,16299.431.amd64fre.rs3_release_svc_escrow.1805...,Home,1.0,0,NaN,0.0,117.0,ExistsNotSet,1.0,1.0,Notebook,Windows.Desktop,4730.0,312820.0,8.0,5.0,2880.0,NaN,953869.0,HDD,940100.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,li-i,0.000000e+00,10.0.16299.492,amd64,rs3_release_svc_escrow,16299,492,Core,CORE,UUPUpgrade,8.0,31,Notify,0,IS_GENUINE,Retail,0.0,0.0,Retail,0.0,556.0,13909.0,1,0.0,0.0,0,0,0.0,0.0,10.0,1
3,5dd6c5c23e09214f43fbe89888bf98f7,win8defender,1.1.15100.1,4.11.15063.0,1.273.912.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,44,NaN,27.0,57.0,218,windows10,x64,10.0.0.0,15063,768,rs2,15063.0.amd64fre.rs2_release.170317-1834,Home,1.0,0,NaN,0.0,105.0,NaN,1.0,1.0,Desktop,Windows.Desktop,1980.0,228125.0,8.0,5.0,2970.0,NaN,476940.0,HDD,476438.0,0,8192.0,Desktop,23.1,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.15063.0,amd64,rs2_release,15063,0,CoreSingleLanguage,CORE

In [8]:
# ==================================================================================================================
# ===========================================> Exploración tipos <==================================================
# ==================================================================================================================

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892148 entries, 0 to 892147
Data columns (total 83 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   MachineIdentifier                                  892148 non-null  object 
 1   ProductName                                        892148 non-null  object 
 2   EngineVersion                                      892148 non-null  object 
 3   AppVersion                                         892148 non-null  object 
 4   AvSigVersion                                       892148 non-null  object 
 5   IsBeta                                             892148 non-null  int64  
 6   RtpStateBitfield                                   888846 non-null  float64
 7   IsSxsPassiveMode                                   892148 non-null  int64  
 8   DefaultBrowsersIdentifier                          43369 non-null   float6

In [9]:
# ==================================================================================================================
# ===========================================> Exploración nulos <==================================================
# ==================================================================================================================

df.isna().sum().sort_values(ascending=False)

PuaMode                                              891925
Census_ProcessorClass                                888417
DefaultBrowsersIdentifier                            848779
Census_IsFlightingInternal                           741049
Census_InternalBatteryType                           633987
Census_ThresholdOptIn                                566865
Census_IsWIMBootEnabled                              566118
SmartScreen                                          317362
OrganizationIdentifier                               275543
SMode                                                 53798
CityIdentifier                                        33024
Wdft_IsGamer                                          30427
Wdft_RegionIdentifier                                 30427
Census_InternalBatteryNumberOfCharges                 27062
Census_FirmwareManufacturerIdentifier                 18377
Census_FirmwareVersionIdentifier                      16083
Census_IsFlightsDisabled                

In [10]:
# ==================================================================================================================
# ===========================================> Exploración valores y estadísticas <=================================
# ==================================================================================================================

stats = []

for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    
stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique Values', 'Missing Values %', 'Biggest Category %', 'Type'])
stats_df.sort_values('Missing Values %', ascending=False).head(20)

,Feature,Unique Values,Missing Values %,Biggest Category %,Type
28,PuaMode,1,99.975004,99.975004,object
41,Census_ProcessorClass,3,99.581796,99.581796,object
8,DefaultBrowsersIdentifier,719,95.138811,95.138811,float64
68,Census_IsFlightingInternal,2,83.063460,83.063460,float64
52,Census_InternalBatteryType,37,71.062985,71.062985,object
71,Census_ThresholdOptIn,2,63.539345,63.539345,float64
75,Census_IsWIMBootEnabled,1,63.455615,63.455615,float64
31,SmartScreen,11,35.572797,48.387151,object
15,OrganizationIdentifier,44,30.885346,46.970794,float64
29,SMode,2,6.030165,93.923542,float64


In [11]:
# ==================================================================================================================
# ===========================================> Procesar valores nulos <=============================================
# ==================================================================================================================

drop_cols_min = 0.6
drop_cols = []

for c in df.columns:
    if df[c].isna().sum() / len(df) > drop_cols_min:
        drop_cols.append(c)
        
for c in drop_cols:
    df = df.drop(c, axis=1)

print(len(drop_cols))
print(drop_cols)

7
['DefaultBrowsersIdentifier', 'PuaMode', 'Census_ProcessorClass', 'Census_InternalBatteryType', 'Census_IsFlightingInternal', 'Census_ThresholdOptIn', 'Census_IsWIMBootEnabled']
